# <span style="color:Blue"> T4a: Basic EELS Processing </span>

# Table of Contents<a name="toc"/>
**[1. Introduction](#intro)**
* [1a. Importing data](#import)
* [1b. Understanding the dimensions of eels data in python](#dim)
* [1c. Dark correction](#dark)

**[2. Basic Fitting](#basic)**
* [2a. Edge mapping ](#edge)
* [2b. LBA fitting](#lba)
* [2c. Checking the quality of the background fit](#qual)

**[3. Basic Quantitative Analysis](#quant)**
* [3a. Fine structure analysis ](#fine)
* [3b. Line Profiles](#line)
* [3c. RGB mapping](#rgb)
* [3d. Saving data](#saving)

<span style="color:orange">**Note: At the end of each section, please check with your group's TA and ask any questions that you might have about what you've just gone through. Take this opportunity to discuss the section with others in your group! Also, feel free to ask questions at any point in this tutorial. Make sure you have a good grasp on what it is you're doing in each section before moving on!**</span>

# <span style="color:purple"> 1. Introduction </span> <a name="intro"/> 

In this notebook we will explore the basics of our EELS analysis python package. While programs like NION Swift are useful for quickly browsing through your data, modules like this are useful for creating quickly reproducable results and performing more in depth analysis.

Let's start out by running the import cell below. This will give us all of the necessary functions for our EELS processing.

## <span style="color:purple">1a. Importing data </span> <a name="import"/> 

In [ ]:
## Imports
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

import eels

For this basic tutorial, we will work through the same data set used in the **Nion Swift** tutorial. This is an EELS data set of **SBMO (SmBaMn$_2$O$_6$)** grown on a **DSO (DyScO$_3$)** substrate. 

It is always important to start by defining file paths for importing and saving data processed in this notebook.

In [ ]:
# file path for directory with data files
path = "../Data/T4a/" 

SIdm = "SmBaMnO_onDSO_0p4step_5ms_750eV_0p5disp_5mmap_50pA_SI.dm4"

Now let's import the EELS data. The **3D data cube** created from an EELS measurement is often denoted as a spectral image or **SI**.

In [ ]:
energy, SI, pxscale, disp, params = eels.specload(path+SIdm)

As you can see above, ***`eels.specload`*** will output the following variables:
 - ***energy*** : energy loss values corresponding to third axis of SI data.
 - ***SI*** : SI data cube
 - ***pxscale*** : real space pixel size 
 - ***disp*** : energy dispersion
 - ***params*** : list of parameters pulled from metadata of .dm3/.dm4 data file

## <span style="color:purple">1b. Understanding the dimensions of eels data in python </span>  <a name="dim"/> 

As mentioned earlier, the **SI has three dimensions**, the first two axes represent where the scan probe was in real space, while the third axis will represent the energy loss of an electron at a given scan position **(X,Y,dE)**.

Let's start by plotting the **sum spectrum** across the entire SI. This is nice for quickly seeing what elements you have in your SI. To do this, we want to sum along all of the real space scan positions, meaning we will **sum the first two axes together**.

We can also create a **2D inelastically scattered image** of our SI data set by summing all of the inelastically scattered electrons collected at each pixel. This is done by **summing along the third axis**. This image is more or less a map of the scattering density across the SI, where thicker regions or regions with stronger scatterers will contain more multiple inelastic scattering events, meaning we will collect more electrons in the spectrometer.

In [ ]:
## Let's start this cell by finding the dimensions of the SI data cube

xdim,ydim,edim = np.shape(SI)

print("The SI is a data cube of the electron counts of a given energy loss on the EELS detector for each (x,y) scan position, with size ("+str(xdim)+","+str(ydim)+","+str(edim)+")")
print("The rawE array is a one dimensional array of energy values corresponding to the 3rd axis in the SI data cube with length "+str(len(energy)))

## If we sum this data cube over the first two dimensions (the scan dimension), we end up with sum energy spectrum of 
## the electron energy loss across the entire scan area

sumspec = np.sum(SI,axis=(0,1))

eels.easyplot("Sum Spectrum")
plt.plot(energy,sumspec)

## If we would like to see an image of the scan area using all of the inelastically scattered electrons detected for each pixel,
## we would sum along the 3rd dimension
## discusion on the contrast inversion from HAADF, brighter pixels where we have lower energy scattering.

SIim = np.sum(SI,axis=2)

plt.matshow(SIim,cmap='gray')
plt.title('Full energy SI')

If you would like to see an individual spectrum at a given scan position, we can plot this by inputting the pixel coordinates in the first two axes of the SI data cube. Remember that in python, the first dimension is the row and the second dimension is the column. In this notebook I will refer to the **row as the x coordinate and the column as the y coordinate**.

We can also obtain a crude image of the scattering at a certain energy loss value by plotting a 2D image of the SI data cube with a single value in the third axis.

You will see exactly how this works in python in the cell below. Feel free to try different scan positions and energy values.

<span style="color:orange">**Note: Maximum allowed (x,y) values are (1025,56). Inputting a larger value will give errors because these values are outside the (x,y) dimensions of the SI. Maximum allowed energy value is 1257.25 eV for the same reason.**</span>

In [ ]:
## If we would like to see the spectrum at a given scan position (here at the x=50,y=50 scan position)
## Check the dimensions of your SI and pick position within the x-y axis.

xpos,ypos = ?,?

eels.easyplot("Energy spectrum at x="+str(xpos)+", y="+str(ypos))
plt.plot(energy,SI[xpos,ypos,:])

## If we would like to see an image of our scan area at a given electron energy loss try:

eloss = ? # this is in eV

chloss = eels.eVtoCh(eloss,energy)
plt.matshow(SI[:,:,chloss],cmap='gray')
plt.title("Scan area with energy loss "+str(eloss)+" eV")

## <span style="color:purple">1c. Dark correction </span> <a name="dark"/> 

Normally we collect a dark reference for the EELS detector as a part of every experiment or acquisition. This is useful for reducing readout noise in the detector. Often, the dark correction is applied before the data is saved, but sometimes it is neccessary to apply the dark correction yourself.

In this case, the data set has already undergone a dark correction, but we will see what the raw data set looks like without this dark correction applied. This data was taken on a direct electron detector, so the dark noise is actaully quite low.

In [ ]:
dark = np.array(hs.load(path+SIdm).original_metadata.ImageList.TagGroup0.ImageTags.EELS.Acquisition.HQ_Dark_Correction.HQ_dark_correction)

rawspec = sumspec + dark*xdim*ydim

eels.easyplot("Dark subtraction")
plt.plot(energy, rawspec/(xdim*ydim), color ='r', label ='Raw data')
plt.plot(energy, dark, color = 'k', label ='Dark spectrum')
plt.plot(energy, sumspec/(xdim*ydim), color = 'b', label ='Corrected data')
plt.legend()

[Return to TOC](#toc)

# <span style="color:purple">2. Basic Fitting </span> <a name="basic"/> 

## <span style="color:purple">2a. Edge mapping </span>  <a name="edge"/> 

We will start by **binning the data set** to shorten our computation time and increase single to noise. This also allows us to perform our most basic fitting routine with high accuracy. We will show some of the more advanced methods of background fitting later in this notebook as well as in part two of this EELS analysis tutorial.

To **bin the SI** we need to define our binning factors in each dimension of the data cube by the convention **`(x factor, y factor, E factor)`** and use **`eels.bin_ndarray(array, bin factors)`**

In [ ]:
bin_factors = (2,2,1)

bin_SI = eels.bin_ndarray(SI[:-1,:,:],bin_factors)

We've already indentified the elements in this data in the Nion Swift portion of this tutorial. Let's use the same background fitting and integration windows you've already found in Swift and check that we can reproduce the maps from earlier.

In python, we need to define an edge parameter variable that contains all of this information. The format of this edge parameter variable will be **`[fit start (eV), fit end (eV), int start (eV), int end (eV), 'Label']`**. Note that the "Label" is just a string used for your own record-keeping. 

In the cell below, enter your edge parameters from Nion Swift for the following elements: Sa, Mn, Ba, & Sm. 

In [ ]:
## Assign params to a variable by edge
## Return to this cell to perform a background subtraction for each edge.

Sc = [??, ??, ??, ??,'Sc-L2,3']
Mn = [??, ??, ??, ??,'Mn-L3,2']
Ba = [??, ??, ??, ??,'Ba-M4,5']
Sm = [??, ??, ??, ??,'Sm-M4,5']

## Define titles used for saving convenience later on
Sc_title = Sc[4]+'_fit'+str(int(Sc[0]))+'-'+str(int(Sc[1]))+'_int'+str(int(Sc[2]))+'-'+str(int(Sc[3]))
Mn_title = Mn[4]+'_fit'+str(int(Mn[0]))+'-'+str(int(Mn[1]))+'_int'+str(int(Mn[2]))+'-'+str(int(Mn[3]))
Ba_title = Ba[4]+'_fit'+str(int(Ba[0]))+'-'+str(int(Ba[1]))+'_int'+str(int(Ba[2]))+'-'+str(int(Ba[3]))
Sm_title = Sm[4]+'_fit'+str(int(Sm[0]))+'-'+str(int(Sm[1]))+'_int'+str(int(Sm[2]))+'-'+str(int(Sm[3]))

Now we will preform a background subtraction on each of these edges. This is done using ***`eels.bgsub_SI`***. The standard way to fit and subtract a power law function to the background is use a log transform followed by a linear least squares fit. This method is very fast and reliable, however, in certain situations it can have drawbacks that we will address later in the tutorial.

For more information on ***`eels.bgsub_SI`***, you can type ***`eels.bgsub_SI?`***

In [ ]:
Sc_bsub = eels.bgsub_SI(bin_SI, energy, Sc,log=True)
Mn_bsub = eels.bgsub_SI(bin_SI, energy, Mn,log=True)
Ba_bsub = eels.bgsub_SI(bin_SI, energy, Ba,log=True)
Sm_bsub = eels.bgsub_SI(bin_SI, energy, Sm,log=True)

Let's look at the edge maps from each of these elements. We can produce these using the function ***`eels.edgemap`***. ***How do these edge maps compare to the edge maps produced in Nion Swift?***

In [ ]:
## Producing edge maps

Sc_edgemap = eels.edgemap(Sc_bsub, energy, Sc)
Mn_edgemap = eels.edgemap(Mn_bsub, energy, Mn)
Ba_edgemap = eels.edgemap(Ba_bsub, energy, Ba)
Sm_edgemap = eels.edgemap(Sm_bsub, energy, Sm)

fig,ax = plt.subplots(1,4,figsize=(6,12))
ax[0].set_title(Sc[4])
ax[0].imshow(Sc_edgemap, cmap='gray')
ax[1].set_title(Mn[4])
ax[1].imshow(Mn_edgemap, cmap='gray')
ax[2].set_title(Ba[4])
ax[2].imshow(Ba_edgemap, cmap='gray')
ax[3].set_title(Sm[4])
ax[3].imshow(Sm_edgemap, cmap='gray')

These maps should look very similar to the maps you've already created in Nion Swift, but now let's see if we can improve upon these maps by implementing some of the additional tools available to us in this EELS package.

## <span style="color:purple">2b. LBA fitting </span> <a name="lba"/> 

We can improve the quality of this background subtraction if we implement **LBA (Local Background Averaging)**. This method applies a gaussian filter to the energy channels in our fit window before applying a fit. You can find more information in ***Data Processing for Atomic Resolution Electron Energy Loss Spectroscopy***, Cueva et al. 2012. [DOI: 10.1017/S1431927612000244](https://www.cambridge.org/core/journals/microscopy-and-microanalysis/article/abs/data-processing-for-atomic-resolution-electron-energy-loss-spectroscopy/B890497A2E7BF967521355C225C23E81)

This is a tool available to us in the ***`eels.bgsub_SI`*** function. 

First we will define a parameter that will set the gaussian FWHM of the filter applied to the fit window energy channels. The size of this window is proportional to the signal to noise ratio improvement in the background window, but it should be noted that too large of a gaussian FWHM will cause artifacts in your edge maps. To see more on this, again you can reference the paper cited above.

In [ ]:
## Now gfwhm is approximately the number of pixels we average the background fit window over to improve fitting.

gfwhm = 15

Sc_bsub_lba = eels.bgsub_SI(bin_SI, energy, Sc,log=True,gfwhm=gfwhm)
Mn_bsub_lba = eels.bgsub_SI(bin_SI, energy, Mn,log=True,gfwhm=gfwhm)
Ba_bsub_lba = eels.bgsub_SI(bin_SI, energy, Ba,log=True,gfwhm=gfwhm)
Sm_bsub_lba = eels.bgsub_SI(bin_SI, energy, Sm,log=True,gfwhm=gfwhm)

Now lets compare the background subtracted edge maps we made earlier to the LBA background data we just produced.

In [ ]:
## Producing edge maps

Sc_edgemap_lba = eels.edgemap(Sc_bsub_lba, energy, Sc)
Mn_edgemap_lba = eels.edgemap(Mn_bsub_lba, energy, Mn)
Ba_edgemap_lba = eels.edgemap(Ba_bsub_lba, energy, Ba)
Sm_edgemap_lba = eels.edgemap(Sm_bsub_lba, energy, Sm)

fig,ax = plt.subplots(1,4,figsize=(6,12))
ax[0].set_title(Mn[4])
ax[0].imshow(Mn_edgemap, cmap='gray')
ax[1].set_title(Mn[4]+ ', LBA ' + str(gfwhm))
ax[1].imshow(Mn_edgemap_lba, cmap='gray')
ax[2].set_title(Ba[4])
ax[2].imshow(Ba_edgemap, cmap='gray')
ax[3].set_title(Ba[4]+ ', LBA ' + str(gfwhm))
ax[3].imshow(Ba_edgemap_lba, cmap='gray')

***Go back and try changing the gfwhm value. How do these edge maps change with various gfwhm values?*** 

## <span style="color:purple">2c. Checking the quality of the background fit </span> <a name="qual"/> 

The quality of these edge maps should have increased after implementing LBA, but how did this happen? To get a clearer picture of why our edge maps improved, we need to be able to check the background subtracted data. In the cells below we will go over basic checks you should do before continuing with quantitative analysis.

**Checking the pre-edge and post-edge of the background subtracted data:**

After performing a background subtraction, the quickest way to check the quality is to plot the summed background subtracted data and checking <span style="color:orange">*two key indicators of a good background subtraction*<span style="color:orange"></span>. 

The <span style="color:orange">*first*</span> is to look at the <span style="color:orange">*"pre-edge"*</span>. This is the region in the spectrum just before the onset of the energy peak that was used to fit the background. If the background was properly fitted, the pre-edge should be flat and centered at zero counts. This would indicate that the power law fit properly matched the background at each pixel. Any deviation from zero would indicate a mismatch between the fitting and the background.

The <span style="color:orange">*second*</span> thing to look at is the <span style="color:orange">*"post-edge"*</span>, or the region just after the peaks of interest. This post-edge region should generally follow some power law decay to zero, indicating that the only remaining background signal data is from the edge of interest. If the post-edge region appears to continually increase above the edge of interest or decrease past zero counts, this would indicate a bad fit.

Let's start by comparing the first attempt at a background subtraction to the second attempt using LBA. This cell is written to study the Mn edge, but ***try rewritting this cell to check all four egdes we studied in this data set.***

In [ ]:
## To assess whole SI background
eels.easyplot("Compare Background subtraction")
plt.plot(energy, sum(sum(Mn_bsub)),label='non-LBA')
plt.plot(energy, sum(sum(Mn_bsub_lba)),label='LBA')
plt.xlim(Mn[0], Mn[3]+250)
plt.axhline(0, color='k', linestyle='--')
plt.legend()

***Can you tell why the LBA edge maps consitently outperformed the non-LBA edge maps?***

**Checking discrete regions of the background subtracted data:**

A more robust way of checking the quality of our background subtraction is to check background subtracted spectrum in various discrete regions of the SI for consitency. The first check is only showing us on average how the background subtraction performed across the entire SI. To get a clearer picture of how this background subtraction is doing across the SI, we can use ***`eels.edge_check(background subtracted data, energy axis, edge)`*** GUI to scan the spectrum across the SI.

Below we show how to perform this check on one of the four edge maps. The SI is also cropped along the 'x axis' for ease of viewing the SI, but this can easily be performed on the entire SI by removing the cropping. ***Try reproducing this check on all four edges. How do the LBA data sets compare to the non-LBA data sets?***

You can sum over discrete regions by toggling either of the two check boxes labled ***`Region 1`*** and ***`Region 2`***. Change the size and shapes of the regions with the sliders on the right of these check boxes. You can adjust the shape moving the sliders or directly typing the parameters in the boxes to the right of the sliders.

**<span style="color:red"> If the notebook is slow, you can shutdown earlier figures by clicking on the power button on the top right of the figure. These GUI figures should be more responsive if the figures not currently being used are made inactive.</span>**

In [ ]:
## This function will show the spectrum summed over discrete regions.

eels.edge_check(Ba_bsub_lba[150:350,:,:],energy,Ba) # Notice how the SI is being cropped here.

**Checking quality of the fit window:**

The checks above are very useful for verifying the quality of a background subtraction after it has already been done. It is a good idea to before running a background subtraction to check the fit window that you will use in the background subtraction. The region chosen for the fit window should be over a continous area of the background, with no obvious changes abrupt changes in the power law like decay. It should also end just prior to the onset of the EELS edge of interest.

To check the fit window, we can use the ***`eels.fit_check(SI,energy axis)`*** GUI. This GUI is similar to the tab used in Nion Swift used to define your fit window. This function can also be used to check individual pixel to see how the background changes across the SI and ensure your fit is consistently matching the background. 

Up until now, we have only used a power law curve to fit the background, but in some select cases, this may not be the optimal function to describe the background. You can check that the type of fit you have decided to use is optimal for your background in this GUI as well. In this eels package, we can use power law as well as linear and exponential fittings, and you can check these fits against the power law fit in this GUI. For this tutorial, the power law is sufficient at fitting our background. 

**You should have already optimized your fit window parameters in Nion Swift, but take some time to double check these fit parameters using the GUI below.**

In [ ]:
eels.fit_check(bin_SI,energy)

**Side Note:**

We have already shown that the ***`eels.edge_check`*** GUI is useful for checking the quality of your background subtracted data, but this GUI can also be used to screen your SI for the elemental distribution in across the SI. Try using the function again, but now on the original SI. Check that the concenctrations we saw earlier in the edge maps follow with the rise and decay of certain edges on either side of the interface in this data set.

Note: In the cell below, the SI has been cropped for convience of viewing the interface of SBMO and DSO.

In [ ]:
eels.edge_check(bin_SI[400:,:,:],energy)

**<span style="color:red"> Before moving onto the next section, ensure that your background subtracted data passes all of the checks we have described above. </span>**

[Return to TOC](#toc)

# <span style="color:purple">3. Basic Quantitative Analysis </span>  <a name="quant"/> 

## <span style="color:purple">3a. Fine structure analysis </span> <a name="fine"/> 

Now that we are confident in the quality of our background subtraction, we can also perform additional quantitative analysis on data set. Let's start by plotting an image of the change in the eels spectrum across the interface.

***Try running this cell on all four of the background subtracted data sets.***

In [ ]:
## We can visualize the change in the eels spectrum across an interface by summing along the axis normal to the interface

# Change these variables to modify the edge we are studying!
edge = Mn
data = Mn_bsub_lba

## integrated profile from left to right
# profile = np.sum(data, axis = 0)

## integrated profile from top to bottom
profile = np.sum(data, axis = 1)

emin = edge[2] - 20.
emax = edge[3] + 10.
chmin = eels.eVtoCh(emin, energy)
chmax = eels.eVtoCh(emax, energy)

fig, ax = plt.subplots(figsize = (5,5))
extent=[emin,emax,np.shape(data)[0],0]
ax.imshow(profile[:,chmin:chmax], extent = extent,cmap = 'gray')
plt.xlabel('Energy loss (eV)')
plt.ylabel('X pixel')
ax.set_aspect(.5)
plt.show()

##  <span style="color:purple">3b. Line Profiles </span> <a name="line"/> 

Additionally, we can plot a line profile of the concentration across the interface by summing the edge map along the axis normal to the interface.

In [ ]:
edgemap = Mn_edgemap_lba


## integrated line profile from left to right
# lineprofile = np.sum(edgemap, axis = 0)

## integrated profile from top to bottom
lineprofile = np.sum(edgemap, axis = 1)


eels.easyplot("line profile of concentration")
plt.plot(np.arange(len(lineprofile))*pxscale,lineprofile)
plt.xlabel('Distance (nm)')
plt.ylabel("Intensity (a.u.)")
plt.show()

***How are the elements distributed through the thickness of the film? Try plotting the concentrations of Sm, Ba, and Mn on the same plot to see if you can reveal any interesting order or disorder.***

##  <span style="color:purple">3c. RGB mapping </span> <a name="rgb"/> 

Now that you have multiple background subtracted data sets, we can also create **compositional maps** of the different elements in the data set.

Let's start by picking three elements of interest. For this data set, we can pick the three elements we have mapped in the SBMO crystal.

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(4,12))
ax[0].set_title(Sm[4])
ax[0].imshow(Sm_edgemap, cmap='gray')
ax[1].set_title(Mn[4])
ax[1].imshow(Mn_edgemap, cmap='gray')
ax[2].set_title(Ba[4])
ax[2].imshow(Ba_edgemap, cmap='gray')

Using ***`eels.easyrgb`*** we can create compostional maps using any of the primary colors: Red, Green, Blue, Cyan, Magenta, and Yellow.

In [ ]:
## Now lets make a composite image of these edge maps.
## Pick the colors red, green, blue, cyan, magenta, and yellow using 'r','g','b','c','m','y' 

rgbmap = eels.easyrgb([Sm_edgemap,Mn_edgemap,Ba_edgemap],['r','g','b'])    
cmymap = eels.easyrgb([Sm_edgemap,Mn_edgemap,Ba_edgemap],['c','m','y'])    

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(8,20))
ax1.set_title('Sm: Red , Mn : Green , Ba : Blue') # change this based on the colors you choose above
ax1.imshow(rgbmap)
ax2.set_title('Sm: Cyan , Mn : Magenta , Ba : Yellow') # change this based on the colors you choose above
ax2.imshow(cmymap)

***Do you see anything unique happening in this SI now that we've created these compositional maps? Is the BMSO completely uniform or can you identify some regions where the crystal seems to become less ordered?***

##  <span style="color:purple">3d. Saving data </span> <a name="saving"/> 

Now that we have gone through the basics of the EELS python package, let's learn how to properly save all of our data! There are three common forms of data that we save.

The first is an **npz** file. This is useful for saving background subtracted data that we have made in this notebook. We can save the data along with the energy axis and edge parameters used. This is great for quickly loading in data that you've previously made in this notebook as well as performing more involved quantitative analysis that we will go over in the next notebook.

The second is a **tiff**. This is great for saving the lovely edge maps you've now made in python as well as the RGB compositional maps.

Lastly, if you'd like to save any of the figures you've made, we typically save those as **png** files.

In the cell below, there is a guide for saving each of these types of files. 

In [ ]:
## NPZ file saving:

## Make sure to include all edge parameters in the title along with any additional fitting tools used such as LBA.
np.savez(path + Sm_title + '_lba_'+str(gfwhm), 
        bsub = Sm_bsub_lba,
        energy = energy, 
        edge = Sm)

## Tiff file saving:

## Saving a 2D edge map:
eels.saveTiff(Sm_edgemap_lba,path+Sc_title + '_lba_'+str(gfwhm)+'_edgemap')

## Saving a RGB compositional map:
eels.saveTiff(np.swapaxes(rgbmap,0,1),path+'RGBmap_'+'Sm_Red__Mn_Green__Ba_Blue')


## PNG file saving:

edgemap = Sm_edgemap_lba
lineprofile = np.sum(edgemap, axis = 1)

eels.easyplot("line profile of concentration")
plt.plot(np.arange(len(lineprofile))*pxscale,lineprofile)
plt.xlabel('Distance (nm)')
plt.ylabel("Intensity (a.u.)")
## This is the important line for saving
plt.savefig(path+Sm_title+'_LBA'+'_conc_h',transparent = 'true', bbox_inches = 'tight', dpi = 300)

# <span style="color:red">End of notebook </span>
[Return to TOC](#toc)